In [1]:
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.cloud import bigquery

# Set the path to the JSON key file
json_key_path = '/Users/ebbewienholts/Downloads/begra-data-8544713e6ca4.json'

# Set the environment variable for authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_key_path

# Create a BigQuery client
client = bigquery.Client()

# Define the BigQuery query
query = """
    SELECT
        query,
        data_date,
        COUNT(*) AS query_count
    FROM
        `begra-data.searchconsole.searchdata_url_impression`
    WHERE
        data_date BETWEEN '2023-06-01' AND '2023-06-05'
    GROUP BY
        query, data_date
    ORDER BY
        data_date ASC
"""

# Run the query
query_job = client.query(query)

# Fetch the results
results = query_job.result()

# Extract the data from the BigQuery result
data = []
for row in results:
    data.append({
        "query": row.query,
        "query_count": row.query_count
    })


In [2]:
data

[{'query': None, 'query_count': 2526},
 {'query': 'kast', 'query_count': 1},
 {'query': 'tego', 'query_count': 10},
 {'query': 'afrol', 'query_count': 1},
 {'query': 'bakje', 'query_count': 3},
 {'query': 'begra', 'query_count': 60},
 {'query': 'brgra', 'query_count': 7},
 {'query': 'fetra', 'query_count': 2},
 {'query': 'hxbxd', 'query_count': 9},
 {'query': 'lxbxh', 'query_count': 13},
 {'query': 'perfo', 'query_count': 1},
 {'query': 'wesco', 'query_count': 6},
 {'query': 'zegel', 'query_count': 2},
 {'query': '140 kg', 'query_count': 1},
 {'query': 'belgra', 'query_count': 5},
 {'query': 'dexion', 'query_count': 5},
 {'query': 'ipe200', 'query_count': 2},
 {'query': 'kasten', 'query_count': 10},
 {'query': 'ladder', 'query_count': 1},
 {'query': 'laskar', 'query_count': 1},
 {'query': 'ligger', 'query_count': 4},
 {'query': 'rongen', 'query_count': 4},
 {'query': 'schoor', 'query_count': 2},
 {'query': 'wielen', 'query_count': 1},
 {'query': '1000x45', 'query_count': 3},
 {'query':

In [3]:
import os
import re
import plotly.graph_objects as go
from google.cloud import bigquery
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Set the path to the JSON key file
json_key_path = '/Users/ebbewienholts/Downloads/begra-data-8544713e6ca4.json'

# Set the environment variable for authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_key_path

# Create a BigQuery client
client = bigquery.Client()

# Define the BigQuery query
query = """
    SELECT
        query,
        CAST(data_date AS DATE) AS data_date,
        COUNT(*) AS query_count
    FROM
        `begra-data.searchconsole.searchdata_url_impression`
    WHERE
        data_date BETWEEN '2023-06-01' AND '2023-06-05'
    GROUP BY
        query, data_date
    ORDER BY
        data_date ASC
"""

# Run the query
query_job = client.query(query)

# Fetch the results
results = query_job.result()

# Extract the data from the BigQuery result
data = []
for row in results:
    data.append({
        "query": row.query,
        "data_date": row.data_date,
        "query_count": row.query_count
    })

# Create a DataFrame from the data
df = pd.DataFrame(data)



In [4]:
df

,query,data_date,query_count
0,None,2023-06-01,2526
1,kast,2023-06-01,1
2,tego,2023-06-01,10
3,afrol,2023-06-01,1
4,bakje,2023-06-01,3
...,...,...,...
5886,stellingkast in elkaar zetten handleiding,2023-06-03,1
5887,staanderbeschermer voor magazijnstellingen,2023-06-03,1
5888,gereedschapskasten magazijnkasten tweedehands,2023-06-03,1
5889,palletstellingen arbeidsomstandighedenbesluit,2023-06-03,1


In [1]:
import os
import plotly.graph_objects as go
from google.cloud import bigquery
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set the path to the JSON key file
json_key_path = '/Users/ebbewienholts/Downloads/begra-data-8544713e6ca4.json'

# Set the environment variable for authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_key_path

# Create a BigQuery client
client = bigquery.Client()

# Define the BigQuery query
query = """
    SELECT
        query,
        CAST(data_date AS DATE) AS data_date,
        COUNT(*) AS query_count
    FROM
        `begra-data.searchconsole.searchdata_url_impression`
    WHERE
        data_date BETWEEN '2023-06-01' AND '2023-06-05'
    GROUP BY
        query, data_date
    ORDER BY
        data_date ASC
"""

# Run the query
query_job = client.query(query)

# Fetch the results
results = query_job.result()

# Extract the data from the BigQuery result
data = []
for row in results:
    data.append({
        "query": row.query,
        "data_date": row.data_date.strftime('%Y-%m-%d'),  # convert to string
        "query_count": row.query_count
    })

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Filter out 'None' from the query list and sort by total search count
total_query_counts = df.groupby('query')['query_count'].sum().sort_values(ascending=False)
queries = total_query_counts.index.tolist()

# Create a combobox (dropdown menu with search)
dropdown = widgets.Combobox(options=queries, description='Query:', ensure_option=True, value=queries[0] if queries else None)

# Create the initial plot
fig = go.FigureWidget()

# Function to update the plot based on the selected query
def update_plot(change):
    query = change.new
    if query in queries:  # only update if the query is in the list
        plot_data = df[df['query'] == query].copy()  # copy to avoid SettingWithCopyWarning
        plot_data['data_date'] = pd.to_datetime(plot_data['data_date'])  # convert to datetime
        fig.data = []
        fig.add_trace(go.Scatter(x=plot_data['data_date'], y=plot_data['query_count'], mode='lines+markers'))
        fig.layout.title.text = f'Query Trends for "{query}"'

# Attach the event handler to the dropdown menu
dropdown.observe(update_plot, names='value')

# Display the dropdown menu
display(dropdown)

# Set the initial plot
update_plot(type('obj', (object,), {'new': dropdown.value}))

# Display the chart
display(fig)


Combobox(value='magazijnstellingen', description='Query:', ensure_option=True, options=('magazijnstellingen', …

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'type': 'scatter',
              'uid': '9…

In [7]:
# Save the chart as an HTML file
fig.write_html('chart.html')


In [3]:
import os
import plotly.graph_objects as go
from google.cloud import bigquery
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import chart_studio.plotly as py
import chart_studio.tools as tls

# Set the path to the JSON key file
json_key_path = '/Users/ebbewienholts/Downloads/begra-data-8544713e6ca4.json'

# Set the environment variable for authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_key_path

# Create a BigQuery client
client = bigquery.Client()

# Define the BigQuery query
query = """
    SELECT
        query,
        CAST(data_date AS DATE) AS data_date,
        COUNT(*) AS query_count
    FROM
        `begra-data.searchconsole.searchdata_url_impression`
    WHERE
        data_date BETWEEN '2023-06-01' AND '2023-06-05'
    GROUP BY
        query, data_date
    ORDER BY
        data_date ASC
"""

# Run the query
query_job = client.query(query)

# Fetch the results
results = query_job.result()

# Extract the data from the BigQuery result
data = []
for row in results:
    data.append({
        "query": row.query,
        "data_date": row.data_date.strftime('%Y-%m-%d'),  # convert to string
        "query_count": row.query_count
    })

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Filter out 'None' from the query list and sort by total search count
total_query_counts = df.groupby('query')['query_count'].sum().sort_values(ascending=False)
queries = total_query_counts.index.tolist()

# Create a combobox (dropdown menu with search)
dropdown = widgets.Combobox(options=queries, description='Query:', ensure_option=True, value=queries[0] if queries else None)

# Create the initial plot
fig = go.FigureWidget()

# Function to update the plot based on the selected query
def update_plot(change):
    query = change.new
    if query in queries:  # only update if the query is in the list
        plot_data = df[df['query'] == query].copy()  # copy to avoid SettingWithCopyWarning
        plot_data['data_date'] = pd.to_datetime(plot_data['data_date'])  # convert to datetime
        fig.data = []
        fig.add_trace(go.Scatter(x=plot_data['data_date'], y=plot_data['query_count'], mode='lines+markers'))
        fig.update_layout(title=f'Query Trends for "{query}"')

# Attach the event handler to the dropdown menu
dropdown.observe(update_plot, names='value')

# Display the dropdown menu
display(dropdown)

# Set the initial plot
update_plot(type('obj', (object,), {'new': dropdown.value}))

# Display the chart
display(fig)




Combobox(value='magazijnstellingen', description='Query:', ensure_option=True, options=('magazijnstellingen', …

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'type': 'scatter',
              'uid': '2…